In [ ]:
import { resolve } from 'node:path';
const wallpapersPath = '/home/kival/Pictures/Wallpapers/';
const aspectRatioMinThreshold = 4 / 3;
const aspectRatioMaxThreshold = 40 / 9;
const minX = 2560;

type ImageResult = { path: string; x: number; y: number; marked?: boolean };

function collectAllImages(dir: string): ImageResult[] {
   const results: ImageResult[] = [];
   for (const file of Deno.readDirSync(dir)) {
      if (file.name === 'Ignore') continue;
      if (file.isDirectory) {
         results.push(...collectAllImages(resolve(dir, file.name)));
      }
      if (file.isFile) {
         const path = resolve(dir, file.name);
         const output = new Deno.Command('identify', {
            args: [path],
         }).outputSync();
         const res = new TextDecoder().decode(output.stdout).trim();
         if (res) {
            const out = res.split('\n');
            // const resolution = out
            //    .find((s) => s.includes('Geometry'))!
            //    .slice(out.find((s) => s.includes('Geometry'))!.indexOf(':') + 1)
            //    .matchAll(/\d+/g)
            //    .toArray()
            //    .map(Number);
            const resolution = out[0]
               .slice(path.length + 1)
               .matchAll(/\d+/g)
               .toArray()
               .map(Number);
            results.push({ path, x: resolution[0], y: resolution[1] });
         }
      }
   }
   return results;
}

function filterImages(res: ImageResult): boolean {
   return (
      res.x / res.y >= aspectRatioMinThreshold &&
      res.x / res.y <= aspectRatioMaxThreshold
   );
}

function copyToWallpapers(path: string) {
   return function (res: ImageResult): ImageResult {
      if (res.x < minX) {
         const scale = Math.ceil(minX / res.x);
         console.log('Upscaling', res.path, 'by', scale);
         new Deno.Command('waifu2x-ncnn-vulkan', {
            args: [
               '-i',
               res.path,
               '-o',
               resolve(
                  path,
                  res.path.includes('hentai') ? 'homework' : 'main',
                  'up_' + res.path.split('/').pop()!
               ),
               '-s',
               scale.toString(),
               '-n',
               '2',
            ],
         }).outputSync();
      } else {
         Deno.copyFileSync(
            res.path,
            resolve(
               path,
               res.path.includes('hentai') ? 'homework' : 'main',
               res.path.split('/').pop()!
            )
         );
      }
      return res;
   };
}


In [ ]:
const images = [
   '/home/kival/Pictures/Artwork/not hentai',
   '/home/kival/Pictures/Artwork/Touhou Project',
].map((x) => ({
   source: x,
   collection: collectAllImages(x),
}));
images.forEach((group) =>
   console.log(group.collection.length, 'collected from', `"${group.source}"`)
);

var n = images
   .find((group) => !group.source.includes('hentai')).collection.length;
var h = images
   .find((group) => group.source.includes('hentai')).collection.length;

console.log(`1 in ${n / h}`);

In [ ]:
images.forEach((group) => {
   group.collection.forEach((item) => (item.marked = filterImages(item)));
   console.log(
      group.collection.filter((item) => item.marked).length,
      'marked to use from',
      `"${group.source}"`
   );
});

var n = images
   .find((group) => !group.source.includes('hentai'))
   .collection.filter((item) => item.marked).length;
var h = images
   .find((group) => group.source.includes('hentai'))
   .collection.filter((item) => item.marked).length;

console.log(`1 in ${n / h}`);


In [ ]:
Deno.removeSync(wallpapersPath, { recursive: true });
Deno.mkdirSync(wallpapersPath);
Deno.mkdirSync(resolve(wallpapersPath, 'main'));
Deno.mkdirSync(resolve(wallpapersPath, 'homework'));
images
   .flatMap((item) => item.collection)
   .filter((item) => item.marked)
   .forEach(copyToWallpapers(wallpapersPath));
console.log('Saved.');


In [ ]:
import { document } from 'jsr:@ry/jupyter-helper';
import * as Plot from 'npm:@observablehq/plot';

await Deno.jupyter.display(
   Plot.plot({
      marks: [
         Plot.dot(
            images.flatMap((group) =>
               group.collection.map((item) => ({
                  ...item,
                  source: group.source,
                  fill: group.source.includes('hentai')
                     ? item.marked
                        ? '#fff'
                        : '#000'
                     : item.marked
                     ? '#08f'
                     : '#000',
               }))
            ),
            {
               x: 'x',
               y: 'y',
               fill: 'fill',
               title: 'path',
            }
         ),
      ],
      grid: '#000',
      labelArrow: 'right',
      subtitle: 'Resolution',
      document,
   })
);
